## Imports

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import shapiro, norm, spearmanr
import sys
from pathlib import Path


project_dir = Path().resolve().parents[2]
sys.path.append(str(project_dir / "code/"))

## Get Data

In [ ]:

# %% ############################# Variables and data
data_dir = project_dir / "data/"
file_name = ""

data = pd.read_excel(data_dir / file_name)


# Change all "." in variable names to "__"
data = data.rename(columns=lambda x: x.replace(".", "__"))

# Reduce dataset to reporter = parent and visit number = 1 
data = data.query("reporter == 'parent' and visnr == 1")

# Filter for reporter and visit
data = data.query("reporter == 'parent' and visnr == 1")

# Filter by sex (1 = Male, 2 = Female)
#sex_filter = input("Filter by sex:") #Filter by sex (1 for male, 2 for female, leave empty for no filter)
#if sex_filter in ['1', '2']:
#    data = data.query(f"sex12 == {sex_filter}")

sdq_vars = [
    'e_sdq.d00149_hyp_sum'
]
sdq_vars = [c.replace(".", "__") for c in sdq_vars]
print("Modified voice variable names:", sdq_vars)

voice_vars = [
    'stimme.f0_sprech_1',
    'stimme.f0_sprech_2',
    'stimme.f0_sprech_3',
    'stimme.f0_sprech_4',
    'stimme.f0_sprech_5',
    'stimme.spl_sprech_1',
    'stimme.spl_sprech_2',
    'stimme.spl_sprech_3',
    'stimme.spl_sprech_4',
    'stimme.spl_sprech_5',
    'stimme.mpt',
    'stimme.jitter',
    'stimme.dsi'
]
voice_vars = [c.replace(".", "__") for c in voice_vars]
print("Modified voice variable names:", voice_vars)

covariates = [
    'age', 
    'sex12', 
    'soz_winkler_2019.d00408_gesamt_score',
    'c_pub_stat.d00077_pub_status',
    'c_anthro_kh.d00040_bmi_sds',
]
covariates = [c.replace(".", "__") for c in covariates]
print("Modified covariate names:", covariates)

# Keep rows where SDQ is complete AND at there are no NAN for the voice features with the biggest overlap (f0_sprech_1)
data = data[
    data[sdq_vars[0]].notnull() &   # SDQ HI must be present
    data["stimme__f0_sprech_1"].notnull()  # must have value for this voice feature
]

# Select relevant columns
selected_columns = [
    "pseudosic", "sgroup", "visnr", "nvis", "sex", "jahr",
    "soz_winkler_2019__d00408_gesamt_status",
    "c_pub_stat__d00077_stimmbruch",
    "c_pub_stat__d00077_stimmbruch_wann"
] + covariates + sdq_vars + voice_vars

data = data[selected_columns].reset_index(drop=True)

data.head()


In [ ]:
# Get the SDQ variable
sdq_var = sdq_vars[0]  # assuming only one SDQ HI variable

# Calculate overlap n between SDQ and each voice variable
overlap_counts = {
    voice_var: data[[sdq_var, voice_var]].dropna().shape[0]
    for voice_var in voice_vars
}

# Convert to DataFrame for readability
overlap_df = pd.DataFrame.from_dict(overlap_counts, orient='index', columns=['n_overlap_with_SDQ'])
overlap_df = overlap_df.sort_values(by='n_overlap_with_SDQ', ascending=False)

# Display and optionally save
print("Overlap counts between SDQ and each voice variable:")
display(overlap_df)

## Descriptive Statistics

### Descriptive analysis

In [ ]:
pd.set_option('display.float_format', '{:.2f}'.format)

desc_stats = data.describe(include='all').transpose()

numeric_columns = data.select_dtypes(include=['number']).columns
desc_stats['median'] = data[numeric_columns].median()  
desc_stats['range'] = data[numeric_columns].max() - data[numeric_columns].min()  

desc_stats['mode'] = data.mode().iloc[0]  
desc_stats['missing_values'] = data.isnull().sum()  
desc_stats['missing_percentage'] = (data.isnull().sum() / len(data)) * 100  

print("Descriptive Statistics:")
display(desc_stats)

#output_file = "descriptive_statistics_new_26.05.2025.xlsx"
#desc_stats.to_excel(output_file, index=True)
#print(f"Descriptive statistics saved to {output_file}")


### Frequency Counts 

In [ ]:
categorical_variables = ["sex", "soz_winkler_2019__d00408_gesamt_status", "sgroup", "c_pub_stat__d00077_stimmbruch"]
numeric_variables = ["jahr", "age_category", "nvis"]  

# Categorize age into whole numbers (5–18 years)
data["age_category"] = data["age"].apply(lambda x: int(np.floor(x)) if not pd.isnull(x) else np.nan)
data = data[data["age_category"].between(5, 18)]  

# Create a dictionary to store frequency tables for export
frequency_tables = {}

# Perform frequency counts and plot for categorical variables
for column in categorical_variables:
    # Frequency counts
    freq = data[column].value_counts()
    frequency_tables[column] = freq  # Store frequency table

    # Display frequency counts
    print(f"\nFrequency counts for {column}:\n")
    print(freq)

    # Plot the frequency counts
    plt.figure(figsize=(10, 6))
    bars = freq.plot(kind='bar', color='skyblue', edgecolor='black')

    # Add numbers on top of the bars
    for bar in bars.patches:
        plt.text(
            bar.get_x() + bar.get_width() / 2,
            bar.get_height(),
            int(bar.get_height()),
            ha='center',
            va='bottom',
            fontsize=12
        )

    plt.title(f"Frequency Counts for {column}", fontsize=16)
    plt.xlabel(column, fontsize=14)
    plt.ylabel("Frequency", fontsize=14)
    plt.xticks(rotation=45, fontsize=12)
    plt.tight_layout()
    plt.savefig(f"frequency_plot_{column}.png", dpi=300)  # Save the plot
    plt.show()

# Perform frequency counts and plot for numeric variables
for column in numeric_variables:
    # Frequency counts
    freq = data[column].value_counts().sort_index()  # Sort by index for numeric order
    frequency_tables[column] = freq  # Store frequency table

    # Display frequency counts
    print(f"\nFrequency counts for {column}:\n")
    print(freq)

    # Plot the frequency counts
    plt.figure(figsize=(10, 6))
    bars = freq.plot(kind='bar', color='lightgreen', edgecolor='black')

    # Add numbers on top of the bars
    for bar in bars.patches:
        plt.text(
            bar.get_x() + bar.get_width() / 2,
            bar.get_height(),
            int(bar.get_height()),
            ha='center',
            va='bottom',
            fontsize=12
        )

    plt.title(f"Frequency Counts for {column}", fontsize=16)
    plt.xlabel(column, fontsize=14)
    plt.ylabel("Frequency", fontsize=14)
    plt.xticks(rotation=45, fontsize=12)
    plt.tight_layout()
    plt.savefig(f"frequency_plot_{column}.png", dpi=300)  # Save the plot
    plt.show()

# Save frequency tables to an Excel file
output_file = "frequency_counts_new_26.05.25.xlsx"
with pd.ExcelWriter(output_file) as writer:
    for column, freq in frequency_tables.items():
        # Ensure sheet name is <= 31 chars and has no invalid characters
        safe_name = f"{column}_Frequency"[:31]
        freq.to_excel(writer, sheet_name=safe_name)

print(f"Frequency counts saved to {output_file}")



### Distribution and Normality Check 

In [ ]:
for column in data.select_dtypes(include=['float64', 'int64']).columns:
    plt.figure(figsize=(6, 4))
    
    # Histogram
    column_data = data[column].dropna()
    plt.hist(column_data, bins=30, alpha=0.7, color='blue', density=True, label='Histogram')

    # Overlay normality line
    mu, std = column_data.mean(), column_data.std()
    x = np.linspace(column_data.min(), column_data.max(), 100)
    plt.plot(x, norm.pdf(x, mu, std), color='red', label='Normal Distribution')

    plt.title(f'Histogram and Normality Line for {column}')
    plt.xlabel(column)
    plt.ylabel('Frequency')
    plt.legend()
    plt.savefig(f"distribution_{column}.png")  
    plt.close()
    
    # Shapiro-Wilk test for normality
    stat, p = shapiro(column_data)
    if p > 0.05:
        print(f"{column}: Data is normally distributed (p = {p:.3f})")
    else:
        print(f"{column}: Data is NOT normally distributed (p = {p:.3f})")

### Correlation Analysis (Spearman-Rho)

In [ ]:
selected_covariates = ['age', 'sex12', 'soz_winkler_2019__d00408_gesamt_score','c_pub_stat__d00077_pub_status','c_anthro_kh__d00040_bmi_sds']

variables_to_correlate = sdq_vars + voice_vars + selected_covariates
spearman_corr = data[variables_to_correlate].corr(method='spearman')

# Calculate significance for each correlation
p_values = pd.DataFrame(np.ones(spearman_corr.shape), columns=spearman_corr.columns, index=spearman_corr.index)
for col1 in variables_to_correlate:
    for col2 in variables_to_correlate:
        if col1 != col2:
            # Align the data for the two columns
            valid_data = data[[col1, col2]].dropna()
            _, p = spearmanr(valid_data[col1], valid_data[col2])
            p_values.loc[col1, col2] = p

# Mask insignificant correlations (p > 0.05)
sig_mask = p_values <= 0.05
annotated_corr = spearman_corr.where(sig_mask, other=np.nan)

# Format p-values as whole decimal numbers
p_values = p_values.applymap(lambda x: f"{x:.3f}" if x >= 0.001 else "<0.001")

# Save the correlation matrix for reference
spearman_corr.to_excel("spearman_correlation_matrix.xlsx", index=True)
print("Spearman Correlation Matrix (filtered variables) saved as 'spearman_correlation_matrix.xlsx'.")

###### Pairwise correlation

# Generate a list of significant correlation pairs with p-values
correlation_list = []
for col1 in variables_to_correlate:
    for col2 in variables_to_correlate:
        if col1 != col2:
             # Drop missing data for the pair
            valid_data = data[[col1, col2]].dropna()
            n = len(valid_data)
            corr_value, p = spearmanr(valid_data[col1], valid_data[col2])
            # Get the correlation coefficient and p-value
            corr_value = spearman_corr.loc[col1, col2]
            p_value = p_values.loc[col1, col2]
            # Append to the list if the correlation is significant (optional filtering)
            correlation_list.append({"Variable 1": col1, "Variable 2": col2, "Correlation": corr_value, "P-value": p_value, "N": n })

# Convert the list to a DataFrame
correlation_pairs_df = pd.DataFrame(correlation_list)

# Drop duplicate pairs (since correlation is symmetric)
correlation_pairs_df = correlation_pairs_df.drop_duplicates(subset=["Variable 1", "Variable 2"], keep="first")

# Sort by absolute correlation (optional)
correlation_pairs_df = correlation_pairs_df.sort_values(by="Correlation", key=lambda x: abs(x), ascending=False)

# Save the correlation list to an Excel file
correlation_pairs_df.to_excel("spearman_correlation_pairs.xlsx", index=False)
print("Correlation pairs with coefficients and p-values saved as 'spearman_correlation_pairs.xlsx'.")

###### Plotting
#Create Heatmap
variable_name_mapping = {
    "e_sdq__d00149_hyp_sum": "SDQ_HI",
    "stimme__f0_sprech_1": "f0_quiet_I",
    "stimme__f0_sprech_2": "f0_conversation_II",
    "stimme__f0_sprech_3": "f0_presentation_III",
    "stimme__f0_sprech_4": "f0_loud_IV",
    "stimme__f0_sprech_5": "f0_quiet_V",
    "stimme__spl_sprech_1": "spl_quiet_I",
    "stimme__spl_sprech_2": "spl_conversation_II",
    "stimme__spl_sprech_3": "spl_presentation_III",
    "stimme__spl_sprech_4": "spl_loud_IV",
    "stimme__spl_sprech_5": "spl_quiet_V",
    "stimme__mpt": "MPT",
    "stimme__jitter": "Jitter",
    "stimme__dsi": "DSI",
    "age": "Age",
    "sex12": "Sex",
    "soz_winkler_2019__d00408_gesamt_score": "SES",
    "c_pub_stat__d00077_pub_status": "Pubertal status",
    "c_anthro_kh__d00040_bmi_sds": 'BMI_SDS',
}

# Rename the rows and columns of the correlation matrix for the heatmap
annotated_corr_renamed = annotated_corr.rename(index=variable_name_mapping, columns=variable_name_mapping)

# Generate a new mask based on the renamed matrix
mask = annotated_corr_renamed.isnull()

# Display the correlation matrix as a heatmap with renamed labels
plt.figure(figsize=(12, 8))
heatmap = sns.heatmap(annotated_corr_renamed, annot=True, fmt='.2f', cmap='coolwarm', cbar=True, mask=mask,
                      annot_kws={"size": 8}, linewidths=0.5, center=0, vmin=-1, vmax=1)
heatmap.xaxis.set_ticks_position('top')
plt.xticks(rotation=90, fontsize=10)
plt.yticks(fontsize=10)
plt.title('Spearman Correlation Matrix (Significant Correlations)', pad=20)
fig = plt.gcf()  # Get current figure
fig.patch.set_facecolor('white')  # Set figure background to white
ax = plt.gca()  # Get current axes
ax.set_facecolor('white')  # Set plot (axes) background to white
plt.savefig("spearman_correlation_heatmap_renamed_.png", dpi=300, bbox_inches='tight')  # Save as PNG
plt.show()


In [ ]:
selected_covariates = ['age', 'sex12', 'soz_winkler_2019__d00408_gesamt_score','c_pub_stat__d00077_pub_status','c_anthro_kh__d00040_bmi_sds']

variables_to_correlate = sdq_vars + voice_vars + selected_covariates
spearman_corr = data[variables_to_correlate].corr(method='spearman')

# Calculate significance for each correlation
p_values = pd.DataFrame(np.ones(spearman_corr.shape), columns=spearman_corr.columns, index=spearman_corr.index)
for col1 in variables_to_correlate:
    for col2 in variables_to_correlate:
        if col1 != col2:
            # Align the data for the two columns
            valid_data = data[[col1, col2]].dropna()
            _, p = spearmanr(valid_data[col1], valid_data[col2])
            p_values.loc[col1, col2] = p

# Mask insignificant correlations (p > 0.05)
sig_mask = p_values <= 0.05
annotated_corr = spearman_corr.where(sig_mask, other=np.nan)

# Format p-values as whole decimal numbers
p_values = p_values.applymap(lambda x: f"{x:.3f}" if x >= 0.001 else "<0.001")

# Save the correlation matrix for reference
spearman_corr.to_excel("spearman_correlation_matrix.xlsx", index=True)
print("Spearman Correlation Matrix (filtered variables) saved as 'spearman_correlation_matrix.xlsx'.")

###### Pairwise correlation

# Generate a list of significant correlation pairs with p-values
correlation_list = []
for col1 in variables_to_correlate:
    for col2 in variables_to_correlate:
        if col1 != col2:
             # Drop missing data for the pair
            valid_data = data[[col1, col2]].dropna()
            n = len(valid_data)
            corr_value, p = spearmanr(valid_data[col1], valid_data[col2])
            # Get the correlation coefficient and p-value
            corr_value = spearman_corr.loc[col1, col2]
            p_value = p_values.loc[col1, col2]
            # Append to the list if the correlation is significant (optional filtering)
            correlation_list.append({"Variable 1": col1, "Variable 2": col2, "Correlation": corr_value, "P-value": p_value, "N": n })

# Convert the list to a DataFrame
correlation_pairs_df = pd.DataFrame(correlation_list)

# Drop duplicate pairs (since correlation is symmetric)
correlation_pairs_df = correlation_pairs_df.drop_duplicates(subset=["Variable 1", "Variable 2"], keep="first")

# Sort by absolute correlation (optional)
correlation_pairs_df = correlation_pairs_df.sort_values(by="Correlation", key=lambda x: abs(x), ascending=False)

# Save the correlation list to an Excel file
correlation_pairs_df.to_excel("spearman_correlation_pairs.xlsx", index=False)
print("Correlation pairs with coefficients and p-values saved as 'spearman_correlation_pairs.xlsx'.")

###### Plotting
#Create Heatmap
variable_name_mapping = {
    "e_sdq__d00149_hyp_sum": "SDQ_HI",
    "stimme__f0_sprech_1": "f0_quiet_I",
    "stimme__f0_sprech_2": "f0_conversation_II",
    "stimme__f0_sprech_3": "f0_presentation_III",
    "stimme__f0_sprech_4": "f0_loud_IV",
    "stimme__f0_sprech_5": "f0_quiet_V",
    "stimme__spl_sprech_1": "spl_quiet_I",
    "stimme__spl_sprech_2": "spl_conversation_II",
    "stimme__spl_sprech_3": "spl_presentation_III",
    "stimme__spl_sprech_4": "spl_loud_IV",
    "stimme__spl_sprech_5": "spl_quiet_V",
    "stimme__mpt": "MPT",
    "stimme__jitter": "Jitter",
    "stimme__dsi": "DSI",
    "age": "Age",
    "sex12": "Sex",
    "soz_winkler_2019__d00408_gesamt_score": "SES",
    "c_pub_stat__d00077_pub_status": "Pubertal status",
    "c_anthro_kh__d00040_bmi_sds": 'BMI_SDS',
}

# Rename the rows and columns of the correlation matrix for the heatmap
annotated_corr_renamed = annotated_corr.rename(index=variable_name_mapping, columns=variable_name_mapping)

# Generate a new mask based on the renamed matrix
mask = annotated_corr_renamed.isnull()

# Display the correlation matrix as a heatmap with renamed labels
plt.figure(figsize=(12, 8))
heatmap = sns.heatmap(annotated_corr_renamed, annot=True, fmt='.2f', cmap='coolwarm', cbar=True, mask=mask,
                      annot_kws={"size": 8}, linewidths=0.5, center=0)
heatmap.xaxis.set_ticks_position('top')
plt.xticks(rotation=90, fontsize=10)
plt.yticks(fontsize=10)
plt.title('Spearman Correlation Matrix (Significant Correlations)', pad=20)
fig = plt.gcf()  # Get current figure
fig.patch.set_facecolor('white')  # Set figure background to white
ax = plt.gca()  # Get current axes
ax.set_facecolor('white')  # Set plot (axes) background to white
plt.savefig("spearman_correlation_heatmap_renamed_.png", dpi=300, bbox_inches='tight')  # Save as PNG
plt.show()


### Collinearity Analysis

In [ ]:
high_correlation = spearman_corr[(spearman_corr > 0.8) | (spearman_corr < -0.8)]
print("Highly Correlated Variables (Collinearity Check):")
display(high_correlation)

#Create list of highly correlated pairs
high_correlation_pairs = []
for col in high_correlation.columns:
    for row in high_correlation.index:
        if row != col and not np.isnan(high_correlation.loc[row, col]):
            corr_value = high_correlation.loc[row, col]
            p_value = p_values.loc[row, col]
            high_correlation_pairs.append((row, col, corr_value, p_value))

high_correlation_list = pd.DataFrame(high_correlation_pairs, columns=['Variable 1', 'Variable 2', 'Correlation', 'P-Value'])
high_correlation_list = high_correlation_list.drop_duplicates().sort_values(by='Correlation', ascending=False)

print("List of Highly Correlated Variable Pairs:")
display(high_correlation_list)

high_correlation.to_excel("high_correlation_matrix.xlsx", index=True)
high_correlation_list.to_excel("high_correlation_list.xlsx", index=False)
print("Highly Correlated Variables saved as 'high_correlation_matrix.xlsx' and 'high_correlation_list.xlsx'.")

### Scatterplots correlations

In [ ]:
sdq_vars 
voice_vars
selected_covariates = ['age', 'sex12', 'soz_winkler_2019__d00408_gesamt_score',
                       'c_pub_stat__d00077_pub_status', 'c_anthro_kh__d00040_bmi_sds']

# Plot SDQ vs Voice variables
for sdq_var in sdq_vars:
    for voice_var in voice_vars:
        # Drop missing values for the pair
        valid_data = data[[sdq_var, voice_var]].dropna()

        # Create scatterplot
        plt.figure(figsize=(8, 6))
        sns.scatterplot(x=valid_data[voice_var], y=valid_data[sdq_var], alpha=0.7, edgecolor=None)
        plt.title(f"Scatterplot: {voice_var} vs {sdq_var}", fontsize=14)
        plt.xlabel(voice_var, fontsize=12)
        plt.ylabel(sdq_var, fontsize=12)
        plt.grid(alpha=0.3)

        plt.savefig(f"{voice_var}_vs_{sdq_var}.png", dpi=300, bbox_inches="tight")
        plt.close()

        print(f"Scatterplot saved: {voice_var}_vs_{sdq_var}.png")

# Plot SDQ vs selected covariates
for sdq_var in sdq_vars:
    for covariate in selected_covariates:
        # Drop missing values for the pair
        valid_data = data[[sdq_var, covariate]].dropna()

        plt.figure(figsize=(8, 6))
        sns.scatterplot(x=valid_data[covariate], y=valid_data[sdq_var], alpha=0.7, edgecolor=None)
        plt.title(f"Scatterplot: {covariate} vs {sdq_var}", fontsize=14)
        plt.xlabel(covariate, fontsize=12)
        plt.ylabel(sdq_var, fontsize=12)
        plt.grid(alpha=0.3)

        plt.savefig(f"{covariate}_vs_{sdq_var}.png", dpi=300, bbox_inches="tight")
        plt.close()

        print(f"Scatterplot saved: {covariate}_vs_{sdq_var}.png")

# Plot Voice variables vs selected covariates
for voice_var in voice_vars:
    for covariate in selected_covariates:
        # Drop missing values for the pair
        valid_data = data[[voice_var, covariate]].dropna()

        # Create scatterplot
        plt.figure(figsize=(8, 6))
        sns.scatterplot(x=valid_data[covariate], y=valid_data[voice_var], alpha=0.7, edgecolor=None)
        plt.title(f"Scatterplot: {covariate} vs {voice_var}", fontsize=14)
        plt.xlabel(covariate, fontsize=12)
        plt.ylabel(voice_var, fontsize=12)
        plt.grid(alpha=0.3)

        # Save scatterplot
        plt.savefig(f"{covariate}_vs_{voice_var}.png", dpi=300, bbox_inches="tight")
        plt.close()

        print(f"Scatterplot saved: {covariate}_vs_{voice_var}.png")

